## Challenge Set 9: Part 3
#### *Intermediate level SQL* ####
Subject:      Soccer  
Date:         11/01/2018  
Name:         Steven Bierer  

----

In [15]:
# Connect to the Soccer database (already downloaded and unzipped) #
import sqlite3
import pandas as pd
pd.set_option("display.max_columns",50)

cnx = sqlite3.connect('supporting_files/soccer_database.sqlite')
cur = cnx.cursor()

In [16]:
# Look at the tables available in the loaded database #
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'", cnx)

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


### Question 1 -- Which team scored the most points when playing at home?
*__Answer__: Real Madrid CF leads the pack with 505 goals scored on their home pitch.*

In [25]:
# First, take a peek at the Match table #
query = ''' SELECT * FROM Match
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,...,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,4.20,1.85,3.2,3.5,1.80,3.3,3.75,None,None,None,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,3.95,1.90,3.2,3.5,1.90,3.2,3.50,None,None,None,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,2.55,2.60,3.1,2.3,2.50,3.2,2.50,None,None,None,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,6.80,1.40,3.9,6.0,1.44,3.6,6.50,None,None,None,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,1.60,4.00,3.3,1.7,4.00,3.4,1.72,None,None,None,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [28]:
# That's the table we need, so now I'll get the total goals scored #
query = ''' SELECT home_team_api_id, team_long_name, SUM(home_team_goal) AS "Total Goals"
            FROM Match
            JOIN Team ON Match.home_team_api_id = Team.team_api_id
            GROUP BY home_team_api_id
            ORDER BY "Total Goals" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,home_team_api_id,team_long_name,Total Goals
0,8633,Real Madrid CF,505
1,8634,FC Barcelona,495
2,9925,Celtic,389
3,9823,FC Bayern Munich,382
4,8640,PSV,370


### Question 2 -- Did this team also score the most points when playing away? 
*__Answer__: No, the team with the most away goals was Getafe CF.*

In [29]:
# Similar to the query for Q1 #
query = ''' SELECT away_team_api_id, team_long_name, SUM(home_team_goal) AS "Total Goals"
            FROM Match
            JOIN Team ON Match.away_team_api_id = Team.team_api_id
            GROUP BY away_team_api_id
            ORDER BY "Total Goals" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,away_team_api_id,team_long_name,Total Goals
0,8305,Getafe CF,278
1,9791,Heracles Almelo,272
2,9904,Hannover 96,270
3,8697,SV Werder Bremen,268
4,10217,ADO Den Haag,268


### Question 3 -- How many matches resulted in a tie? 
*__Answer__: 6,596 games resulted in a tie, out of 25,979 matches listed.*

In [94]:
# Just count the home team goal entries when they equal the away team goals #
# Note: There are no null score values, but I checked just in case.
query = ''' SELECT COUNT(home_team_goal) AS "Tie Games"
            FROM Match
            WHERE home_team_goal = away_team_goal AND home_team_goal IS NOT NULL; '''
pd.read_sql_query(query, cnx)

,Tie Games
0,6596


In [34]:
# Count how many total matches there were #
pd.read_sql_query('SELECT COUNT(*) FROM MATCH;', cnx)

,COUNT(*)
0,25979


### Question 4 -- How many players have Smith for a last name? How many have 'smith' anywhere in their name? 
*__Answer__: 14 players have the last name of Smith, while 17 have "smith" somewhere in their names (the "Nays" have it).*

In [46]:
# Process last names from the Player table #
query = ''' SELECT player_name FROM Player
            WHERE player_name LIKE '% Smith';'''
pd.read_sql_query(query, cnx)

,player_name
0,Adam Smith
1,Alan Smith
2,Brad Smith
3,Cameron Smith
4,Chris Smith
5,Daan Smith
6,David Smith
7,Gordon Smith
8,Graeme Smith
9,Graeme Smith


In [101]:
# Same approach, but expand the fitting criteria of "smith" #
query = ''' SELECT player_name FROM Player
            WHERE player_name LIKE '%smith%' AND player_name LIKE '%Smith%';'''
pd.read_sql_query(query, cnx)

,player_name
0,Adam Smith
1,Alan Smith
2,Brad Smith
3,Cameron Smith
4,Chris Smith
5,Daan Smith
6,David Smith
7,Gary Naysmith
8,Gordon Smith
9,Graeme Smith


### Question 5 -- What was the median tie score? Use the value determined in the previous question for the number of tie games.
*__Answer__: It appears that the median tie score was 1-1.*

In [93]:
# We know the number of tie games is even, so it's OK to dispense with the fanciness to calculate a median #
query = ''' SELECT AVG(home_team_goal)
            FROM (SELECT home_team_goal
                FROM Match
                WHERE home_team_goal = away_team_goal
                ORDER BY home_team_goal
                LIMIT 2
                OFFSET (6596 / 2)); '''
pd.read_sql_query(query, cnx)

,AVG(home_team_goal)
0,1.0


### Question 5 -- What percentage of players prefer their left or right foot?
*__Answer__: 75% prefer their right foot.*

In [110]:
# First, it's good to be aware of the options (maybe some "prefer" both feet) #
query = ''' SELECT DISTINCT preferred_foot
            FROM Player_Attributes; '''
pd.read_sql_query(query, cnx)

,preferred_foot
0,right
1,left
2,None


In [116]:
# Including all options, determine what percentage prefer their right foot (the others will be "left" or "none") #
query = ''' SELECT COUNT(*)*100/(select COUNT(*) from Player_Attributes) AS 'Percent Right Foot'
            FROM Player_Attributes
            WHERE preferred_foot = 'right'; '''
pd.read_sql_query(query, cnx)

,Percent Right Foot
0,75


In [103]:
# Process last names from the Player table #
query = ''' SELECT *
            FROM Player_Attributes
            WHERE preferred_foot = 'left'; '''
pd.read_sql_query(query, cnx)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,6,189615,155782,2016-04-21 00:00:00,74,76,left,high,medium,80,53,58,71,40.0,73,70.0,69,68,71,79,78,78.0,67,90.0,71,85.0,79,56,62,68,67,60,66.0,59,76,75,78.0,14,7,9,9,12
1,7,189615,155782,2016-04-07 00:00:00,74,76,left,high,medium,80,53,58,71,32.0,73,70.0,69,68,71,79,78,78.0,67,90.0,71,85.0,79,56,60,68,67,60,66.0,59,76,75,78.0,14,7,9,9,12
2,8,189615,155782,2016-01-07 00:00:00,73,75,left,high,medium,79,52,57,70,29.0,71,68.0,69,68,70,79,78,78.0,67,90.0,71,84.0,79,56,59,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
3,9,189615,155782,2015-12-24 00:00:00,73,75,left,high,medium,79,51,57,70,29.0,71,68.0,69,68,70,79,78,78.0,67,90.0,71,84.0,79,56,58,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
4,10,189615,155782,2015-12-17 00:00:00,73,75,left,high,medium,79,51,57,70,29.0,71,68.0,69,68,70,79,78,78.0,67,90.0,71,84.0,79,56,58,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
5,11,189615,155782,2015-10-16 00:00:00,73,77,left,high,medium,79,51,57,70,29.0,71,68.0,69,68,70,79,78,78.0,67,90.0,71,84.0,79,56,58,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
6,12,189615,155782,2015-09-25 00:00:00,74,78,left,high,medium,79,51,57,70,29.0,71,68.0,69,68,70,80,78,78.0,67,90.0,71,84.0,79,56,58,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
7,13,189615,155782,2015-09-21 00:00:00,73,77,left,medium,medium,79,51,57,70,29.0,67,68.0,69,68,68,79,78,78.0,67,90.0,71,84.0,79,56,58,67,66,58,65.0,59,76,75,78.0,14,7,9,9,12
8,14,189615,155782,2015-01-09 00:00:00,71,75,left,medium,medium,78,50,56,69,28.0,66,67.0,68,67,67,79,82,79.0,71,90.0,70,84.0,79,50,56,66,65,57,64.0,58,73,72,72.0,13,6,8,8,11
9,15,189615,155782,2014-12-05 00:00:00,71,77,left,medium,medium,78,50,56,69,28.0,66,67.0,68,67,67,79,82,79.0,71,90.0,70,84.0,79,50,56,66,65,57,64.0,58,73,72,72.0,13,6,8,8,11
